## Тема проекта. Государственные деньги для НКО

### Описание и цель проекта
НКО хотят повысить свои шансы на получение грантов/госконтрактов от государства. Для этого им важно понять, что именно может повлиять на это. Результаты исследования можно использовать для того, чтобы подсказывать НКО, как повысить шансы на финансовую поддержку.

Цель — спрогнозировать вероятность получения грантов/госконтрактов от государства для организации в зависимости от её характеристик:
* от региона регистрации организации;
* от возраста организации;
* от экономической деятельности организации; 
* от организационно-правовой формы (ОПФ/ОКОПФ);
* от формы НКО;
* от наличия записей организации в соц.сетях (её публичности).

### Постановка задачи

В нашем распоряжении есть дамп данных обо всех НКО России (актуален на 26.08.2021), в котором
содержится информация о получении государственных грантов,
госконтрактов и субсидий, регионе и дате регистрации, а также ОКВЭД
(классификатор экономической деятельности).

Необходимо проверить, есть ли зависимость вероятности получения
грантов/госконтрактов от государства от региона регистрации, возраста, экономической деятельности, а так же, от организационно-правовой формы, от формы НКО и от наличия записей организации в соц.сетях (её публичности).

### Структура проекта
1. Сбор данных
2. Анализ и обработка данных
3. Визуализация данных
4. Кодирование признаков
5. Решение задачи классификации
6. Подготовка модели к продакшену

In [1]:
# Подключение необходимых библиотек
import pandas as pd
import ijson
import warnings
warnings.filterwarnings('ignore')

1. Сбор данных

In [2]:
files = [
    'ngo_dump_01.json',
    'ngo_dump_02.json',
    'ngo_dump_03.json',
    'ngo_dump_04.json',
    'ngo_dump_05.json',
    'ngo_dump_06.json',
    'ngo_dump_07.json',
    'ngo_dump_08.json',
    'ngo_dump_09.json',
    'ngo_dump_10.json',
    'ngo_dump_11.json',
    'ngo_dump_12.json',
    'ngo_dump_13.json',
    'ngo_dump_14.json',
    'ngo_dump_15.json'
]

# Список ключей для считывания данных из json-файла
colunms_for_fd = [
    'ogrn', # ОГРН организации (уникальный номер, присваиваемый организации при её регистрации, и он всегда неизменен) 
    'shortName', # сокращенное наименование организации 
    'regionCode', # код региона организации
    'regionName', # название региона организации
    'dateReg', # дата регистрации организаций - указывается только для организаций, зарегистрированных до 2002 года
    'dateOgrn', # дата присвоения ОГРН - начиная с 2002 года
    'dateLiquid', #  дата ликвидации юрлица
    'originDate', # дата происхождения
    'egrulStatus', # статус организации на основании данных ЕГРЮЛ. Варианты: Действует, Ликвидирована
    'opf', # сведения об организационно-правовой форме (ОПФ) организации на основании данных ЕГРЮЛ    
    'mainOkved', # сведения об основном ОКВЭД
    'addOkved', # дополнительный ОКВЭД    
    'incomeDetail', # детализация сведений о доходах
    'incomeTotal', # общая сумма доходов из известных источников - контрактов, договоров, президентских грантов, федеральных субсидий    
    'hasRegionalSupport', # наличие сведений о том, что организация была получателем региональной поддержки в качестве СО НКО 
                            # (Социально Ориентированная Некоммерческая Организация)   
    'minjustForm', # форма НКО, согласно системе Минюста РФ - УНРО    
    'minjustStatus', # статус НКО в системе Минюста РФ - УНРО. Варианты: Исключена, Действует 
    'socialMedia' # сведения об учетных записях организации в соцсетях    
]

In [3]:
# Считываем дамп данных
data = []
for filename in files:
    with open('./openngo_2021_26_08/' + filename, 'r') as f:
        objects = ijson.items(f, '', multiple_values=True)        
        for row in objects:        
            r = []
            for col in colunms_for_fd:
                r.append(row.get(col, 0)) # вернёт 0 в случае, если данного ключа не существует            
                data.append(r) 
df = pd.DataFrame(data, columns=colunms_for_fd)
#df = df.loc[1:1000,:]
display(df.head())

ogrn shortName regionCode       regionName dateReg  \
1  1207800141790     "АМА"         78  Санкт-Петербург    None   
2  1207800141790     "АМА"         78  Санкт-Петербург    None   
3  1207800141790     "АМА"         78  Санкт-Петербург    None   
4  1207800141790     "АМА"         78  Санкт-Петербург    None   
5  1207800141790     "АМА"         78  Санкт-Петербург    None   

                                dateOgrn dateLiquid  \
1  {'$date': '2020-10-30T00:00:00.000Z'}       None   
2  {'$date': '2020-10-30T00:00:00.000Z'}       None   
3  {'$date': '2020-10-30T00:00:00.000Z'}       None   
4  {'$date': '2020-10-30T00:00:00.000Z'}       None   
5  {'$date': '2020-10-30T00:00:00.000Z'}       None   

                              originDate egrulStatus  \
1  {'$date': '2020-10-30T00:00:00.000Z'}   Действует   
2  {'$date': '2020-10-30T00:00:00.000Z'}   Действует   
3  {'$date': '2020-10-30T00:00:00.000Z'}   Действует   
4  {'$date': '2020-10-30T00:00:00.000Z'}   Действует   
5  {'$date': '2020-10-30T00:00:00.000Z'}   Действует   

                                                 opf  \
1  {'name': 'Ассоциации (союзы)', 'code': '20600'...   
2  {'name': 'Ассоциации (союзы)', 'code': '20600'...   
3  {'name': 'Ассоциации (союзы)', 'code': '20600'...   
4  {'name': 'Ассоциации (союзы)', 'code': '20600'...   
5  {'name': 'Ассоциации (союзы)', 'code': '20600'...   

                                           mainOkved addOkved  \
1  {'name': 'Деятельность профессиональных членск...       []   
2  {'name': 'Деятельность профессиональных членск...       []   
3  {'name': 'Деятельность профессиональных членск...       []   
4  {'name': 'Деятельность профессиональных членск...       []   
5  {'name': 'Деятельность профессиональных членск...       []   

                                        incomeDetail incomeTotal  \
1  {'grants': {'totalCount': 0, 'totalSum': 0}, '...           0   
2  {'grants': {'totalCount': 0, 'totalSum': 0}, '...           0   
3  {'grants': {'totalCount': 0, 'totalSum': 0}, '...           0   
4  {'grants': {'totalCount': 0, 'totalSum': 0}, '...           0   
5  {'grants': {'totalCount': 0, 'totalSum': 0}, '...           0   

   hasRegionalSupport                                     minjustForm  \
1               False  Объединения (союз, ассоциация) юридических лиц   
2               False  Объединения (союз, ассоциация) юридических лиц   
3               False  Объединения (союз, ассоциация) юридических лиц   
4               False  Объединения (союз, ассоциация) юридических лиц   
5               False  Объединения (союз, ассоциация) юридических лиц   

      minjustStatus                                        socialMedia  
1  Зарегистрирована  {'youtube': None, 'vk': None, 'facebook': None...  
2  Зарегистрирована  {'youtube': None, 'vk': None, 'facebook': None...  
3  Зарегистрирована  {'youtube': None, 'vk': None, 'facebook': None...  
4  Зарегистрирована  {'youtube': None, 'vk': None, 'facebook': None...  
5  Зарегистрирована  {'youtube': None, 'vk': None, 'facebook': None...

Преобразуем некоторые столбцы, данные в которых, являются списками или словарями

In [4]:
df['dateReg'] = df['dateReg'].apply(lambda x: x if x==None else pd.to_datetime(dict(x).get('$date', 0)).date())
df['dateOgrn'] = df['dateOgrn'].apply(lambda x: x if x==None else pd.to_datetime(dict(x).get('$date', 0)).date())
df['dateLiquid'] = df['dateLiquid'].apply(lambda x: x if x==None else pd.to_datetime(dict(x).get('$date', 0)).date())
df['originDate'] = df['originDate'].apply(lambda x: x if x==None else pd.to_datetime(dict(x).get('$date', 0)).date())

df['opf_name'] = df['opf'].apply(lambda x: x if x==None else dict(x).get('name', 0))
df['opf_code'] = df['opf'].apply(lambda x: x if x==None else dict(x).get('code', 0))
df['opf_version'] = df['opf'].apply(lambda x: x if x==None else dict(x).get('version', 0))

df['mainOkved_name'] = df['mainOkved'].apply(lambda x: dict(x).get('name', 0))
df['mainOkved_code'] = df['mainOkved'].apply(lambda x: dict(x).get('code', 0))

df['sumGrants'] = df['incomeDetail'].apply(lambda x: dict(x).get('grants', 0).get('totalSum', 0))
df['sumFedSubsidies'] = df['incomeDetail'].apply(lambda x: dict(x).get('fedSubsidies', 0).get('totalSum', 0))
df['sumContracts'] = df['incomeDetail'].apply(lambda x: dict(x).get('contracts44', 0).get('totalSum', 0)+dict(x).get('contracts223', 0).get('totalSum', 0)+dict(x).get('contracts94', 0).get('totalSum', 0))

df['SM_youtube'] = df['socialMedia'].apply(lambda x: dict(x).get('youtube', 0))
df['SM_vk'] = df['socialMedia'].apply(lambda x: dict(x).get('vk', 0))
df['SM_facebook'] = df['socialMedia'].apply(lambda x: dict(x).get('facebook', 0))
df['SM_twitter'] = df['socialMedia'].apply(lambda x: dict(x).get('twitter', 0))
df['SM_instagram'] = df['socialMedia'].apply(lambda x: dict(x).get('instagram', 0))
df['SM_ok'] = df['socialMedia'].apply(lambda x: dict(x).get('ok', 0))

In [5]:
def funcAddOkved_name(arg):
    '''принимает на вход список arg переменной длины (его элементы - словари), 
       возвращает строку содержащую наименование дополнительных ОКВЭД, разделенных ;'''
    out = str('')
    if str(arg)=='[]':
        out = 'undefined'
    else:
        for item in arg:                  
            if out == str(''): 
                out = item.get('name', 0)
            else:
                out += ';' + item.get('name', 0)
    return out

def funcAddOkved_code(arg):
    '''принимает на вход список arg переменной длины (его элементы - словари),
       возвращает строку содержащую коды дополнительных ОКВЭД, разделенных ;'''
    out = str('')
    if str(arg)=='[]':
        out = 'undefined'
    else:
        for item in arg:                  
            if out == str(''): 
                out = item.get('code', 0)
            else:
                out += ';' + item.get('code', 0)
    return out

df['addOkved_name'] = df['addOkved'].apply(funcAddOkved_name)
df['addOkved_code'] = df['addOkved'].apply(funcAddOkved_code)

# Удаляем столбцы, из которых были сформированы новые признаки
df = df.drop(['opf', 'incomeDetail', 'socialMedia', 'mainOkved', 'addOkved'], axis=1)

Удалим дублирующиеся записи

In [6]:
print('Размерность датафрейма до удаления дубликатов: ', df.shape)
df=df.drop_duplicates()
print('Размерность датафрейма после удаления дубликатов: ', df.shape)

Размерность датафрейма до удаления дубликатов:  (1000, 29)
Размерность датафрейма после удаления дубликатов:  (56, 29)


Сохраним датафрейм в файл csv, это позволит нам не производить каждый раз операции по считыванию данных из файлов и преобразования столбцов, которые
занимают много времени. В случае необходимости, датафрейм для дальнейшей работы можно считывать напрямую из файла my_df.csv

In [7]:
df.to_csv('my_df.csv', index=False)
df = pd.read_csv('my_df.csv')
display(df.head())

ogrn                    shortName  regionCode       regionName  \
0  1207800141790                        "АМА"          78  Санкт-Петербург   
1  1207700349152        ФОНД "УНИВЕРСАЛИСТ М"          77           Москва   
2  1207800011430                          NaN          78  Санкт-Петербург   
3  1207700348976          ТСН "ЛУГИНИНО ПАРК"          77           Москва   
4  1207800081500  АНО "ЦЕНТР "ПЕТЕРБУРГФОРУМ"          78  Санкт-Петербург   

   dateReg    dateOgrn dateLiquid  originDate egrulStatus  incomeTotal  ...  \
0      NaN  2020-10-30        NaN  2020-10-30   Действует          0.0  ...   
1      NaN  2020-09-23        NaN  2020-09-23   Действует          0.0  ...   
2      NaN  2020-02-03        NaN  2020-02-03   Действует          0.0  ...   
3      NaN  2020-09-23        NaN  2020-09-23   Действует          0.0  ...   
4      NaN  2020-07-08        NaN  2020-07-08   Действует          0.0  ...   

   sumFedSubsidies sumContracts SM_youtube SM_vk  SM_facebook SM_twitter  \
0              0.0            0        NaN   NaN          NaN        NaN   
1              0.0            0        NaN   NaN          NaN        NaN   
2              0.0            0        NaN   NaN          NaN        NaN   
3              0.0            0        NaN   NaN          NaN        NaN   
4              0.0            0        NaN   NaN          NaN        NaN   

  SM_instagram SM_ok                                      addOkved_name  \
0          NaN   NaN                                          undefined   
1          NaN   NaN  Издание книг;Издание газет;Виды издательской д...   
2          NaN   NaN                                          undefined   
3          NaN   NaN  Аренда и управление собственным или арендованн...   
4          NaN   NaN  Исследование конъюнктуры рынка и изучение обще...   

                               addOkved_code  
0                                  undefined  
1                  58.11;58.13;58.19;59.20.1  
2                                  undefined  
3                                68.20;68.10  
4  73.20;58.11.1;93.29.9;85.41.9;88.99;70.21  

[5 rows x 29 columns]